### 감성 분석(Sentiment Analysis)
- 문서 내 텍스트가 나타내는 여러 가지 단어 및 문맥을 기반으로 감성 수치를 계산하여 분석
    - 지도학습은 학습 데이터와 타겟 레이블 값을 기반으로 감성 분석 학습을 수행한 뒤 이를 기반으로 다른 데이터의 감성 분석을 예측
    - 비지도학습은 'Lexicon'으로 불리는 감성 어휘 사전을 이용하여 문서의 긍정적, 부정적 감성 여부를 판단

 ### 지도학습 기반 감성 분석 실습 - IMDB 영화 리뷰
 - 영화 리뷰의 텍스트를 분석해 감성 분석 결과가 긍정 혹은 부정인지 예측하는 모델 생성
 - https://www.kaggle.com/c/word2vec-nlp-tutorial/data

In [1]:
import pandas as pd

review_df = pd.read_csv('dataset/labeledTrainData.tsv', header=0,sep='\t',quoting=3)
review_df.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [3]:
print(review_df['review'][0])

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

[과제] 상기 데이터로 감성분석을 수행하세요
- pipeline, tfidf, lr 사용
- stop_words = 'english', ngram_range=(1,2), C=10
- 평가는 accuracy, roc_auc

In [11]:
import re

# <br> html 태그는 replace 함수로 공백으로 변환
review_df['review'] = review_df['review'].str.replace('<br />', ' ')

# 파이썬의 re 모듈을 이용해 영어 문자열이 아닌 문자는 모두 공백으로 변환
review_df['review'] = review_df['review'].apply(lambda x : re.sub("[^a-zA-Z]", " ", x))

# [^a-zA-Z] 는 영어 대/소문자가 아닌 모든 문자를 찾는 정규 표현식

In [12]:
from sklearn.model_selection import train_test_split

# 결정 값 클래스 sentiment 칼럼을 별도로 추출하여 결정 값 데이터 세트화
class_df = review_df['sentiment']

# 원본 데이터 세트에서 id, sentiment 삭제하여 피처 데이터 세트화
feature_df = review_df.drop(['id','sentiment'], axis=1, inplace=False)

# train_test_split() 활용하여 데이터 분리(학습용, 테스트용)
X_train, X_test, y_train, y_test = train_test_split(feature_df, class_df, test_size=0.3, random_state=156)

X_train.shape, X_test.shape
# 학습용 데이터 17500개, 테스트용 데이터 7500개의 리뷰로 구성되어있다

((17500, 1), (7500, 1))

In [ ]:
# 감상평(Review) text를 피처 벡터화한 후 ML 분류 알고리즘을 적용해 예측 성능을 측정

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

# 스톱 워드는 English, filtering, ngram은 (1,2)로 설정해 CountVectorization수행. 
# LogisticRegression의 C는 10으로 설정. 
pipeline = Pipeline([
    ('cnt_vect', CountVectorizer(stop_words='english', ngram_range=(1,2) )),
    ('lr_clf', LogisticRegression(C=10, max_iter = 500))]) # C 값이 작으면 훈련을 덜 복잡 (강한 규제)

# Pipeline 객체를 이용하여 fit(), predict()로 학습/예측 수행. predict_proba()는 roc_auc때문에 수행.  
pipeline.fit(X_train['review'], y_train)
pred = pipeline.predict(X_test['review'])
pred_probs = pipeline.predict_proba(X_test['review'])[:,1]

print('예측 정확도는 {0:.4f}, ROC-AUC는 {1:.4f}'.format(accuracy_score(y_test ,pred),
                                         roc_auc_score(y_test, pred_probs)))

In [14]:
# 스톱 워드는 english, filtering, ngram은 (1,2)로 설정해 TF-IDF 벡터화 수행. 
# LogisticRegression의 C는 10으로 설정. 
pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english', ngram_range=(1,2) )),
    ('lr_clf', LogisticRegression(C=10, max_iter = 500))])

pipeline.fit(X_train['review'], y_train)
pred = pipeline.predict(X_test['review'])
pred_probs = pipeline.predict_proba(X_test['review'])[:,1]

print('예측 정확도는 {0:.4f}, ROC-AUC는 {1:.4f}'.format(accuracy_score(y_test ,pred),
                                         roc_auc_score(y_test, pred_probs)))

예측 정확도는 0.4919, ROC-AUC는 0.5052


In [17]:
# Pipeline을 통해서 텍스트를 벡터화시키고 모델 학습시키기!
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, roc_auc_score
import re
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
# tsv 파일도 csv호출함수로 불러올 수 있음!
# headers=0이면 헤더값(칼럼이름) 출력, 1이면 헤더값 없이 바로 출력
# quoting=3 이면 ""(큰따음포) 인용구는 무시하고 출력함!
review_df = pd.read_csv('./dataset/labeledTrainData.tsv', encoding='utf-8',
                       header=0, sep='\t', quoting=3)
# 정규표현식으로 HTML태그와 숫자 삭제하기
# Series의 replace함수로 HTML태그 없애주기
review_df['review'] = review_df['review'].str.replace('<br />',' ')
# re.sub('패턴','뭘로바꿀지',바꿀문자열)
review_df['review'] = review_df['review'].apply(lambda x : re.sub('[^a-zA-Z]',' ',x))

# 학습, 테스트 데이터 분리
target = review_df['sentiment']
feature = review_df['review']

X_train, X_test, y_train, y_test = train_test_split(feature,
                                                   target,
                                                   test_size=0.3,
                                                   random_state=42)
# Vectorizer인자로는 불용어 제거, ngram범위 설정해주기
pipeline = Pipeline([('cnt_vect',CountVectorizer(stop_words='english',
                                               ngram_range=(1,2))),
                    ('lr_clf', LogisticRegression(C=10))])
# 위에서 설정해준 Pipleline으로 데이터 학습시킬 때는 벡터화시키기 전의 원본 데이터
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
# '1' label로 예측될 확률 추출 for roc_auc_score
y_pred_proba = pipeline.predict_proba(X_test)[:,1]
acc = accuracy_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred_proba)
print(f"정확도:{acc : .3f}\nAUC score:{auc : .3f}") 

정확도: 0.885
AUC score: 0.951


### 비지도 학습기반 감성분석
- 다음은 감성어휘 사전을 이용해 비지도 학습에 기반한 방법이다. 여기서 감성어휘 사전이란, 어떤 단어가 긍정, 부정, 중립의 의미를 얼마나 지니고 있는지 백과사전 형태처럼 만들어 놓은 것을 말한다. 이를 기반으로 주어진 텍스트의 감성을 분석한다.<br><br>

- 감성 어휘 사전 기반의 감성분석에는 SentiWordNet, VADER, Pattern 등 방법이 존재한다. 이번 포스팅에서는 VADER에 대한 예시만 다룰 예정이다. 참고로 Pattern은 파이썬 3.x 버전부터는 호환이 되지 않는다고 한다.<br><br>

- VADER는 소셜 미디어의 감성 분석 용도로 만들어졌다. NLTK에서 제공해주는 SentimentIntensityAnalyzer 객체를 통해 쉽게 감성 분석을 제공한다. 
    - 해당 클래스는 polarity_score() 메서드를 호출해 긍정, 부정, 중립 감성 지수를 구하고 이 3가지 지수를 적절히 조합해 계산한 총 감성 지수까지 계산하게 된다. 
    - 또한 총 감성 지수는 -1에서 1사이의 값으로 표현된다.(총 감성 지수는 compound라고 되어 있는 key값의 value이다.)<br><br>

- 위에서 계산된 compound(총 감성 지수) 값이 사전에 설정한 임계값을 기준으로 임계값보다 높으면 긍정 감성, 낮으면 부정감성으로 판단한다. 보통 임계값은 0.1로 설정하며 경우에 따라 임계값을 조정해 예측 성능을 조절하기도 한다.

In [19]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')

senti_analyzer = SentimentIntensityAnalyzer()
# polarity_score로 하나의 텍스트에 대해 각 부정/객관/긍정 그리고 총 합한 감성지수 출력(-1 ~ 1사이의 값을 가짐)
senti_score = senti_analyzer.polarity_scores(review_df['review'][0])
print(senti_score)

# 해당 코드를 출력하면 아래와 같은 dictionary 형태의 데이터가 보이는데, 여기서 compound의 value값이 총 감성 지수이다.

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\HOME\AppData\Roaming\nltk_data...


{'neg': 0.13, 'neu': 0.743, 'pos': 0.127, 'compound': -0.7943}


In [20]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score
from sklearn.metrics import recall_score, f1_score

# 임계치설정(보통 0.1)을 통해 compound(총 감성지수)가 임계치값보다 높으면 긍정, 낮으면 부정으로 분석
def get_sentiment(review, threshold):
    analyzer = SentimentIntensityAnalyzer()
    scores = analyzer.polarity_scores(review)
    
    compound_score = scores['compound']
    final_sentiment = 1 if compound_score >= threshold else 0
    return final_sentiment

 # 각 텍스트 데이터에 위에서 설정한 감성 label 얻는 함수 적용하기
# 임계값은 0.1로 설정
review_df['vader_pred'] = review_df['review'].apply(lambda x : get_sentiment(x, 0.1))
# 원본 데이터에서 주어진 정답 label과 VADER로 예측한 label 비교
y_target = review_df['sentiment']
y_pred = review_df['vader_pred']

print(confusion_matrix(y_target, y_pred))
print("정확도 :", accuracy_score(y_target, y_pred))
print("정밀도 :", precision_score(y_target, y_pred))
print("재현율 :", recall_score(y_target, y_pred))
print("F1 score :", f1_score(y_target, y_pred))

[[ 6747  5753]
 [ 1858 10642]]
정확도 : 0.69556
정밀도 : 0.6491003354681305
재현율 : 0.85136
F1 score : 0.7365980273403703
